<a href="https://colab.research.google.com/github/Hubert26/suicides_IPPAN/blob/main/ml_analize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Liblaries and settings

In [103]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
import os

!pip install -U dtreeviz
import dtreeviz

from google.colab import files

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# avoid "Arial font not found warnings"
import logging
logging.getLogger('matplotlib.font_manager').setLevel(level=logging.CRITICAL)

np.random.seed(42)

#Function definitions

##plot_confusion_matrix

In [104]:
def plot_confusion_matrix(cm, model):
    # klasyfikacja binarna
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index),
                                      colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(
        width=500,
        height=500,
        title=model+' Confusion Matrix',
        font_size=16,
        template='plotly_dark'
        )
    fig.show()
    return fig

##plot_roc_curve

**pogrubiony tekst**
 Receiver Operating Characteristic
1. True Positive Rate (TPR) = FP / (FP+TN)
2. False Positive Rate (FPR) = TP / (TP+FN)

In [105]:
def plot_roc_curve(y_test, y_pred, model):
    # Binary classification
    from sklearn.metrics import roc_curve
    fpr, tpr, tresh = roc_curve(y_test, y_pred, pos_label=1)

    fig = go.Figure(
        data=[
            go.Scatter(x=fpr, y=tpr,
                    line_color='red',
                    name='ROC Curve'),
            go.Scatter(x=[0, 1], y=[0, 1],
                    mode='lines',
                    line_dash='dash',
                    line_color='#F012BE')
        ],
        layout=go.Layout(xaxis_title='False Positive Rate',
                         yaxis_title='True Positive Rate',
                         title = model + ' ROC Curve',
                         showlegend=False,
                         template='plotly_dark',
                         width=700))
    fig.show()
    return fig


##model_validation


Precision (precyzja) = TP / (TP+FP): Ile obserwacji przewidzianych jako pozytywne jest w rzeczywistości pozytywne.

Recall (czułość) = TP / (TP+FN): Ile obserwacji z wszystkich pozywtywnych sklasyfikowaliśmy jako pozytywne.

F1-score: ważona średnia harmonicza między precyzją a czułością;

Support (wsparcie): liczba próbek, które należą do każdej z klas;

Accuracy (dokładność) = (TP+TN) /(TP+TN+FP+FN))

FPR (False Positive Rate) = FP / (FP+TN) [type I error]

FNR (False Negative Rate) = FN / (FN+TP) [type II error]

In [106]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def model_validation(y_test, y_pred, n_bootstraps=1000):
    # Sprawdzenie, czy y_test i y_pred nie są puste
    if y_test is None or y_pred is None or len(y_test) == 0 or len(y_pred) == 0:
        raise ValueError("y_test and y_pred must not be empty.")

    # Obliczenie miar jakości modelu
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_test, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)
    roc_score = roc_auc_score(y_test, y_pred)

    # Obliczenie macierzy pomyłek
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    # Obliczenie False Positive Rate (FPR) i False Negative Rate (FNR)
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

    # Obliczenie swoistości (specificity)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Utworzenie ramki danych z wynikami
    model_results = pd.DataFrame({
        'recall': [recall],
        'specificity': [specificity],
        'accuracy': [accuracy],
        'precision': [precision],
        'f1': [f1],
        'AUROC': [roc_score],
        'fpr': [fpr],
        'fnr': [fnr]
    })


    return #model_results


##plot_learning_curve

In [107]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):

    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

#Data exploration

In [108]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/suicides_IPPAN/main/data/out_exploration_suicides.csv', delimiter=',', low_memory=False, index_col=False, dtype={'DateY': str, 'DateM': str,})


In [109]:
df_raw = df_raw.dropna(subset=['DateY'])

In [110]:
df_raw['DateY'].unique()

array(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2023'], dtype=object)

In [111]:
df_raw['DateM'].unique()

array(['05', '06', '10', '11', '12', '03', '04', '07', '08', '09', '01',
       '02'], dtype=object)

In [112]:
df_raw.shape

(127034, 27)

In [113]:
df_raw.head(5)

,Income,Age1,Fatal,Method,DateM,Gender,Education,AbuseInfo,DateY,ID_samobójcy,WorkInfo,Substance,Age2,Date,Place,CountContext,Marital,Context_Finances,Context_CloseDeath,Context_FamilyConflict,Context_Disability,Context_HeartBreak,Context_Crime,Context_SchoolWork,Context_MentalHealth,Context_Other,Context_HealthLoss
0,Dependent,07-12,0.0,Self-harm,05,F,Primary,Not,2013,"114771204,00",Student,Alco,00-18,5.2013,Forest,1.0,Single,0,0,0,0,0,0,0,0,1,0
1,Dependent,07-12,0.0,Other,06,F,Pre-primary,Not,2013,"114815301,00",Student,Sober,00-18,6.2013,Forest,1.0,Single,0,0,1,0,0,0,0,0,0,0
2,Dependent,07-12,1.0,Hanging,10,F,Pre-primary,Not,2013,"114936744,00",Student,Sober,00-18,10.2013,House,1.0,Single,0,0,0,0,0,0,0,0,0,0
3,Dependent,07-12,0.0,Other,10,M,Primary,Not,2013,"114944870,00",Student,Sober,00-18,10.2013,Other,1.0,Single,0,0,1,0,0,0,0,0,0,0
4,Dependent,07-12,1.0,Hanging,10,F,Pre-primary,Not,2013,"114948230,00",Student,Sober,00-18,10.2013,House,1.0,Single,0,0,0,0,0,0,0,0,0,0


##Filters and file name

['07-12',
 '13-18',
 '19-24',
 '25-29',
 '30-34',
 '35-39',
 '40-44',
 '45-49',
 '50-54',
 '55-59',
 '60-64',
 '65-69',
 '70-74',
 '75-79',
 '80-84',
 '85+']

 sorted(list(set(df_raw['Age1'])))

['00-18', '19-34', '35-64', '65+']

 sorted(list(set(df_raw['Age2'])))

['F', 'M']

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

sorted(list(set(df_raw['DateY'])))

In [114]:
df_raw['Age'] = df_raw['Age2']

In [115]:
age_group = ['00-18']
gender = ['F']
year = sorted(list(set(df_raw['DateY'])))

In [116]:
age_group

['00-18']

In [117]:
age_group_file_title = "age_group_[" + ''.join(filter(str.isdigit, age_group[0][:3])) + "-" + ''.join(filter(str.isdigit, age_group[-1][-3:])) + "]"
year_group_file_title = "year_[" + str(year[0]) + '-' + str(year[-1]) + "]"

In [118]:
file_title = age_group_file_title + '_' + "gender_" + str(gender) + "_" + year_group_file_title
file_title

"age_group_[00-18]_gender_['F']_year_[2013-2023]"

##Data and NaN exploration

In [119]:
df_data = df_raw[df_raw['Age2'].isin(age_group) & df_raw['Gender'].isin(gender) & df_raw['DateY'].isin(year)]

In [120]:
df_data.shape

(7361, 28)

In [121]:
df_data['Fatal'].value_counts()

Fatal
0.0    6907
1.0     454
Name: count, dtype: int64

In [122]:
##Brakujące dane
total = df_data.isnull().sum().sort_values(ascending=False)
percent = 100*(df_data.isnull().sum()/df_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [123]:
missing_data.head(5)

,Total,Percent
Income,0,0.0
Age1,0,0.0
Context_HealthLoss,0,0.0
Context_Other,0,0.0
Context_MentalHealth,0,0.0


##Dropping columns

In [124]:
df_data.drop(['ID_samobójcy', 'Date'], inplace=True,	 axis=1, errors='ignore')

In [125]:
df_data.drop(['DateM', 'DateY'], inplace=True,	 axis=1, errors='ignore')

In [126]:
df_data.drop(['Age1', 'Age2'], inplace=True,	 axis=1, errors='ignore')

In [127]:
df_data.drop(['Age', 'Gender'], inplace=True,	 axis=1, errors='ignore')

In [128]:
#df_data.drop(['Method'], inplace=True,	 axis=1, errors='ignore')

#CLASSIFIERS

##Dummies

In [129]:
data_dummies = pd.get_dummies(df_data)

In [130]:
data_dummies.shape

(7361, 57)

In [131]:
data_dummies = data_dummies.astype(np.uint8)

In [132]:
data_dummies.head()

,Fatal,CountContext,Context_Finances,Context_CloseDeath,Context_FamilyConflict,Context_Disability,Context_HeartBreak,Context_Crime,Context_SchoolWork,Context_MentalHealth,Context_Other,Context_HealthLoss,Income_Benefits,Income_Dependent,Income_NoSteady,Income_Steady,Method_Drowning,Method_Drugs,Method_Gas,Method_Hanging,Method_Jumping,Method_Other,Method_Poisoning,Method_Schooting,Method_Self-harm,Method_Vehicle,Education_Pre-primary,Education_Primary,Education_Secondary,Education_Vocational,AbuseInfo_Alco,AbuseInfo_Alco&OtherSub,AbuseInfo_Not,AbuseInfo_OtherSub,WorkInfo_Employed,WorkInfo_Student,WorkInfo_Unemployed,Substance_Alco,Substance_Alco&OtherSub,Substance_OtherSub,Substance_Sober,Place_Forest,Place_House,Place_Institution,Place_Isolation,Place_Other,Place_PoliceArmy,Place_Railway,Place_Road,Place_School,Place_UtilitySpaces,Place_WaterRes,Place_Work,Marital_Cohabitant,Marital_Married,Marital_Single,Marital_Single
0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


##Train and test split

In [133]:
Y = data_dummies['Fatal']
X = data_dummies.drop('Fatal', axis=1, errors='ignore')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

##Wages

In [134]:
from sklearn.utils.class_weight import compute_class_weight

# Obliczenie wag klas
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Utworzenie słownika wag klas
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

#DTC

In [135]:
# Inicjalizacja modelu Random Forest z ręcznie ustawionymi wagami klas
dtc = DecisionTreeClassifier(class_weight=class_weight_dict, random_state=42)

# Dopasowanie modelu do danych
dtc.fit(X_train, y_train)

DecisionTreeClassifier(class_weight={0: 0.5334437771795403,
                                     1: 7.975232198142415},
                       random_state=42)

##Grid Search

In [136]:
# Definicja siatki parametrów do przeszukiwania
param_grid = {
    'max_depth': [None],  # Maksymalna głębokość drzewa
    'min_samples_split': [10],  # Minimalna liczba próbek do podziału węzła
    'min_samples_leaf': [10]  # Minimalna liczba próbek w liściu
}

# Inicjalizacja przeszukiwania
grid_search = GridSearchCV(dtc, param_grid=param_grid, cv=5)

# Przeszukiwanie przestrzeni parametrów
grid_search.fit(X_train, y_train)

# Najlepsze znalezione parametry
best_params = grid_search.best_params_
print("Najlepsze parametry:", best_params)

# Ostateczna ocena modelu na zbiorze testowym
accuracy = grid_search.score(X_test, y_test)
print("Dokładność na zbiorze testowym:", accuracy)

Najlepsze parametry: {'max_depth': None, 'min_samples_leaf': 10, 'min_samples_split': 10}
Dokładność na zbiorze testowym: 0.8601177003168855


##Decision Tree Classifier

In [137]:
dtc = DecisionTreeClassifier(
                              class_weight=class_weight_dict,
                              random_state=42,
                              max_depth=best_params['max_depth'],
                              min_samples_split=best_params['min_samples_split'],
                              min_samples_leaf=best_params['min_samples_leaf'])

dtc.fit(X_train, y_train)
dtc_y_pred = dtc.predict(X_test)

##Confusion Matrix

In [138]:
cm = confusion_matrix(y_test, dtc_y_pred)
cm_df = pd.DataFrame(cm, columns=["pred_0", "pred_1"], index=["true_0", "true_1"])
fig_cm = plot_confusion_matrix(cm,'Decision Tree')

In [139]:
dtc_cm_df = cm_df

##Clasification report

In [140]:
report = classification_report(y_test, dtc_y_pred, output_dict=True)

# Konwersja słownika do DataFrame
report_df = pd.DataFrame(report).transpose()
report_df = report_df.loc[["0", "1"]]

In [141]:
dtc_report_df = report_df

##ROC Curve

In [142]:
fig_roc = plot_roc_curve(y_test, dtc_y_pred, 'Decision Tree')

##Validation

In [143]:
dtc_mv = model_validation(y_test, dtc_y_pred)

In [144]:
class_weights_df = pd.DataFrame([class_weight_dict])
class_weights_df.columns = ['class_0_weight', 'class_1_weight']

In [145]:
train_test_info_df = pd.DataFrame({
    'Samples': [df_data.shape[0]],
    'Columns': [df_data.shape[1]],
    'Dummies': [data_dummies.shape[1]],
    'Fatal_0': [df_data['Fatal'].value_counts().get(0, 0)],
    'Fatal_1': [df_data['Fatal'].value_counts().get(1, 0)],
    'train_samples': [X_train.shape[0]],
    'test_samples': [X_test.shape[0]],
    'y_train_0': y_train.value_counts()[0],
    'y_train_1': y_train.value_counts()[1],
    'y_test_0': y_test.value_counts()[0],
    'y_test_1': y_test.value_counts()[1]
})

In [146]:
params_df = pd.concat([train_test_info_df, class_weights_df, pd.DataFrame([best_params])], axis=1).T
params_df

,0
Samples,7361
Columns,20
Dummies,57
Fatal_0,6907
Fatal_1,454
train_samples,5152
test_samples,2209
y_train_0,4829
y_train_1,323
y_test_0,2078


In [147]:
dtc_params_df = params_df

In [148]:
importances = dtc.feature_importances_

# Sortowanie cech według ważności
sorted_indices = importances.argsort()[::-1]
data = {'Feature': [X.columns[index] for index in sorted_indices],
        'Importance': [importances[index] for index in sorted_indices]}

importances_df = pd.DataFrame(data)

In [149]:
importances_df

,Feature,Importance
0,Method_Hanging,0.511651
1,Place_Railway,0.164029
2,Method_Jumping,0.101210
3,Method_Poisoning,0.031426
4,Place_House,0.017706
5,Method_Drugs,0.015696
6,Substance_Sober,0.015297
7,Education_Primary,0.015085
8,Education_Secondary,0.013953
9,Method_Other,0.013186


In [150]:
dtc_importances_df = importances_df

##Atributes

In [162]:
# Pobranie atrybutów drzewa
tree = dtc.tree_

# Pobranie ścieżki decyzji dla każdej próbki
node_indicator = dtc.decision_path(X_train)
leaf_indices = dtc.apply(X_train)

# Funkcja do obliczania głębokości węzłów
def compute_node_depths(tree):
    def get_depth(node_id, current_depth):
        depths[node_id] = current_depth
        if tree.children_left[node_id] != -1:
            get_depth(tree.children_left[node_id], current_depth + 1)
        if tree.children_right[node_id] != -1:
            get_depth(tree.children_right[node_id], current_depth + 1)

    depths = np.zeros(tree.node_count, dtype=np.int32)
    get_depth(0, 0)
    return depths

# Obliczenie głębokości węzłów
node_depths = compute_node_depths(tree)

# Lista do przechowywania informacji o węzłach
nodes_info = []


# Iterowanie przez węzły
for node_id in range(tree.node_count):
    # Pobranie wskaźnika Gini
    gini = tree.impurity[node_id]

    # Pobranie liczby próbek w węźle
    samples = tree.n_node_samples[node_id]

    # Pobranie wartości dla każdej klasy
    values = tree.value[node_id][0]

    # Pobranie liczby próbek nieważonych dla każdej klasy
    unweighted_samples = samples_per_class_in_node[node_id]

    # Pobranie głębokości węzła
    depth = node_depths[node_id]

    # Sprawdzenie, czy węzeł jest liściem
    is_leaf = (tree.children_left[node_id] == -1) and (tree.children_right[node_id] == -1)

    # Pobranie cechy użytej do podziału (jeśli węzeł nie jest liściem)
    feature = tree.feature[node_id] if not is_leaf else -1

    # Pobranie progu podziału (jeśli węzeł nie jest liściem)
    threshold = tree.threshold[node_id] if not is_leaf else -1

    # Utworzenie słownika z informacjami o węźle
    node_info = {
        'Node ID': node_id,
        'Gini': gini,
        'Samples': samples,
        'Values (Weighted)': values,
        'Sum of Values': sum(values),
        'Class Counts (Unweighted)': np.round(values / class_weights),
        'Depth': depth,
        'Is Leaf': is_leaf,
        'Feature': feature,
        'Threshold': threshold,
        'Children Left': tree.children_left[node_id],
        'Children Right': tree.children_right[node_id]
    }

    # Dodanie słownika do listy
    nodes_info.append(node_info)

# Utworzenie DataFrame z listy słowników
dtc_nodes_info_df = pd.DataFrame(nodes_info)

##Leaf

In [155]:
def get_leaf_info(dtc, feature_names=None):
    # Ekstrakcja parametrów drzewa
    tree_ = dtc.tree_
    feature = tree_.feature
    threshold = tree_.threshold
    value = tree_.value
    impurity = tree_.impurity
    n_node_samples = tree_.n_node_samples
    children_left = tree_.children_left
    children_right = tree_.children_right

    leaf_info = []

    def recurse(node, path, leaf_id=0):
        nonlocal leaf_info

        if children_left[node] == children_right[node]:  # Jeśli jest to liść
            class_counts = value[node][0]
            leaf_class = np.argmax(class_counts)
            leaf_details = {
                'node_id': node,
                'leaf_id': leaf_id,
                'path': path,
                'samples_per_class': class_counts.tolist(),
                'leaf_class': leaf_class,
                'gini': impurity[node]
            }
            leaf_info.append(leaf_details)
            leaf_id += 1  # Increment leaf_id for the next leaf
        else:
            if feature_names is not None:
                feature_name = feature_names[feature[node]]
            else:
                feature_name = feature[node]

            recurse(children_left[node], path + [(feature_name, '<=', threshold[node])], leaf_id)
            recurse(children_right[node], path + [(feature_name, '>', threshold[node])], leaf_id)

    recurse(0, [])

    return pd.DataFrame(leaf_info)

In [156]:
# Uzyskaj nazwy cech
dtc_feature_names = X_train.columns.tolist()
# Uzyskanie informacji o liściach
leaf_info_df = get_leaf_info(dtc, dtc_feature_names)

In [157]:
leaf_info_df

,node_id,leaf_id,path,samples_per_class,leaf_class,gini
0,9,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[8.001656657693104, 31.90092879256966]",1,3.206352e-01
1,10,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[6.401325326154484, 0.0]",0,6.772360e-15
2,18,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[84.81756057154703, 0.0]",0,0.000000e+00
3,26,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[10.135431766411266, 0.0]",0,0.000000e+00
4,27,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[51.210602609235885, 0.0]",0,0.000000e+00
5,28,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[20.270863532822517, 15.95046439628483]",0,4.928864e-01
6,30,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[13.869538206668048, 0.0]",0,0.000000e+00
7,31,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[36.274176848208704, 23.925696594427244]",0,4.789619e-01
8,32,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[45.87616483744046, 0.0]",0,0.000000e+00
9,33,0,"[(Method_Hanging, <=, 0.5), (Place_Railway, <=...","[56.011596603851764, 0.0]",0,-3.241851e-14


## Decission Tree graph

###dtreeviz

In [53]:
viz_model = dtreeviz.model(dtc,
                           X_train = X_train,
                           y_train = y_train,
                           feature_names=list(X_train.columns),
                           class_names={0:'NotFatal',1:'Fatal'},
                           target_name='suicides')
v = viz_model.view(orientation='LR', leaftype='barh')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names



###graphviz

In [54]:
# Eksportowanie drzewa do pliku DOT
export_graphviz(dtc, out_file='TREE_graphviz.dot', feature_names=X_train.columns, filled=True, rounded=True, class_names={0:'NotFatal',1:'Fatal'}, special_characters=True, impurity=True, node_ids=True, rotate=True)

# Konwersja pliku DOT na obraz
with open('TREE_graphviz.dot') as f:
    dot_graph = f.read()

#RFC

In [55]:
from sklearn.ensemble import RandomForestClassifier

# Inicjalizacja modelu Random Forest z ręcznie ustawionymi wagami klas
rfc = RandomForestClassifier(class_weight=class_weight_dict, random_state=42)

# Dopasowanie modelu do danych
rfc.fit(X_train, y_train)


RandomForestClassifier(class_weight={0: 0.5334437771795403,
                                     1: 7.975232198142415},
                       random_state=42)

##Grid Search

In [56]:
# Definicja siatki parametrów do przeszukiwania
param_grid = {
    'n_estimators': [100],
    'max_features': ['sqrt'], # √(n_features) cech przy każdym podziale węzła
    'max_depth': [None],  # Maksymalna głębokość drzewa
    'min_samples_split': [10],  # Minimalna liczba próbek do podziału węzła
    'min_samples_leaf': [10]  # Minimalna liczba próbek w liściu
}

# Inicjalizacja przeszukiwania
grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=5)

# Przeszukiwanie przestrzeni parametrów
grid_search.fit(X_train, y_train)

# Najlepsze znalezione parametry
best_params = grid_search.best_params_
print("Najlepsze parametry:", best_params)

# Ostateczna ocena modelu na zbiorze testowym
accuracy = grid_search.score(X_test, y_test)
print("Dokładność na zbiorze testowym:", accuracy)

Najlepsze parametry: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 100}
Dokładność na zbiorze testowym: 0.8967858759619738


##Random Forest Classifier

In [57]:
# Inicjalizacja modelu Random Forest z ustawionymi parametrami
rfc = RandomForestClassifier(
    class_weight=class_weight_dict,
    random_state=42,
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features']
)

# Dopasowanie modelu do danych treningowych
rfc.fit(X_train, y_train)

# Przewidywanie etykiet dla danych testowych
rfc_y_pred = rfc.predict(X_test)

##Confusion Matrix

In [58]:
cm = confusion_matrix(y_test, rfc_y_pred)
cm_df = pd.DataFrame(cm, columns=["pred_0", "pred_1"], index=["true_0", "true_1"])
fig_cm = plot_confusion_matrix(cm,'Random Forest')

In [59]:
rfc_cm_df = cm_df

##Clasification report

In [60]:
report = classification_report(y_test, rfc_y_pred, output_dict=True)

# Konwersja słownika do DataFrame
report_df = pd.DataFrame(report).transpose()
report_df = report_df.loc[["0", "1"]]

In [61]:
rfc_report_df = report_df

##ROC Curve

In [62]:
fig_roc = plot_roc_curve(y_test, rfc_y_pred, 'Random Forest')

##Validation

In [63]:
rfc_mv = model_validation(y_test, rfc_y_pred)

In [64]:
class_weights_df = pd.DataFrame([class_weight_dict])
class_weights_df.columns = ['class_0_weight', 'class_1_weight']

In [65]:
train_test_info_df = pd.DataFrame({
    'Samples': [df_data.shape[0]],
    'Columns': [df_data.shape[1]],
    'Dummies': [data_dummies.shape[1]],
    'Fatal_0': [df_data['Fatal'].value_counts().get(0, 0)],
    'Fatal_1': [df_data['Fatal'].value_counts().get(1, 0)],
    'train_samples': [X_train.shape[0]],
    'test_samples': [X_test.shape[0]],
    'y_train_0': y_train.value_counts()[0],
    'y_train_1': y_train.value_counts()[1],
    'y_test_0': y_test.value_counts()[0],
    'y_test_1': y_test.value_counts()[1]
})

In [66]:
params_df = pd.concat([train_test_info_df, class_weights_df, pd.DataFrame([best_params])], axis=1).T
params_df

,0
Samples,7361
Columns,20
Dummies,57
Fatal_0,6907
Fatal_1,454
train_samples,5152
test_samples,2209
y_train_0,4829
y_train_1,323
y_test_0,2078


In [67]:
rfc_params_df = params_df

In [68]:
importances = rfc.feature_importances_

# Sortowanie cech według ważności
sorted_indices = importances.argsort()[::-1]
data = {'Feature': [X.columns[index] for index in sorted_indices],
        'Importance': [importances[index] for index in sorted_indices]}

importances_df = pd.DataFrame(data)

In [69]:
importances_df

,Feature,Importance
0,Method_Hanging,0.312960
1,Method_Drugs,0.227239
2,Method_Self-harm,0.155906
3,Place_Railway,0.039198
4,Method_Vehicle,0.029329
5,Place_House,0.026071
6,Method_Jumping,0.025504
7,Context_MentalHealth,0.025300
8,Place_School,0.016415
9,Method_Other,0.013722


In [70]:
rfc_importances_df = importances_df

#Saveing data

In [71]:
!ls

sample_data  TREE_graphviz.dot


In [72]:
for file in os.listdir():
    if os.path.isfile(file):
        os.remove(file)

####Saveing params

In [73]:
file_name = "MV_" + file_title + ".xlsx"

In [74]:
with pd.ExcelWriter(file_name, engine='openpyxl') as writer:

    # Zapisz DataFrame params_df
    rfc_params_df.to_excel(writer, sheet_name='RFC Model params', index=True)
    # Zapisz DataFrame importances_df
    rfc_importances_df.to_excel(writer, sheet_name='RFC Importances', index=False)
    # Zapisz DataFrame report_df
    rfc_report_df.to_excel(writer, sheet_name='RFC Classification report', index=True)
    # Zapisz DataFrame cm_df
    rfc_cm_df.to_excel(writer, sheet_name='RFC Confusion Matrix', index=True)
    # Zapisz Model Validation
    rfc_mv.to_excel(writer, sheet_name='RFC Model Validation', index=True)

    # Zapisz DataFrame params_df
    dtc_params_df.to_excel(writer, sheet_name='DTC Model params', index=True)
    # Zapisz DataFrame importances_df
    dtc_importances_df.to_excel(writer, sheet_name='DTC Importances', index=False)
    # Zapisz DataFrame report_df
    dtc_report_df.to_excel(writer, sheet_name='DTC Classification report', index=True)
    # Zapisz DataFrame cm_df
    dtc_cm_df.to_excel(writer, sheet_name='DTC Confusion Matrix', index=True)
    # Zapisz Model Validation
    dtc_mv.to_excel(writer, sheet_name='DTC Model Validation', index=True)
    # Zapisz DataFrame nodes_info_df
    dtc_nodes_info_df.to_excel(writer, sheet_name='DTC Nodes Info', index=True)

# Pobierz plik Excel
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

####Saveing tree dtreeviz

In [75]:
file_name = "TREE_dtreeviz_" + file_title + ".svg"
v.save(file_name)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [76]:
# viz_model.ctree_leaf_distributions()
# plt.tight_layout()
# plt.show()

In [77]:
# # Zapisywanie figury jako pliku HTML
# file_name = "DTC_ROC_CURVE_" + file_title + ".html"
# fig_roc.write_html(file_name)

# # Pobieranie pliku HTML
# files.download(file_name)

####Saveing tree graphviz

In [78]:
file_name = "TREE_graphviz_" + file_title
graphviz.Source(dot_graph)
graphviz.Source(dot_graph).render(file_name, format="svg")
files.download(file_name + '.svg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

####Saveing ROC